In [1]:
import pandas as pd

In [2]:
# read the raw csv
df = pd.read_csv('../data/raw/all.csv')

print(df.shape)
print(df.head())

(249, 11)
             name alpha-2 alpha-3  country-code     iso_3166-2   region  \
0     Afghanistan      AF     AFG             4  ISO 3166-2:AF     Asia   
1   Åland Islands      AX     ALA           248  ISO 3166-2:AX   Europe   
2         Albania      AL     ALB             8  ISO 3166-2:AL   Europe   
3         Algeria      DZ     DZA            12  ISO 3166-2:DZ   Africa   
4  American Samoa      AS     ASM            16  ISO 3166-2:AS  Oceania   

        sub-region intermediate-region  region-code  sub-region-code  \
0    Southern Asia                 NaN        142.0             34.0   
1  Northern Europe                 NaN        150.0            154.0   
2  Southern Europe                 NaN        150.0             39.0   
3  Northern Africa                 NaN          2.0             15.0   
4        Polynesia                 NaN          9.0             61.0   

   intermediate-region-code  
0                       NaN  
1                       NaN  
2               

In [3]:
# select only the columns needed
df_region = df[['name', 'alpha-2', 'region', 'sub-region']]

print(df_region.shape)
print(df_region.head())

(249, 4)
             name alpha-2   region       sub-region
0     Afghanistan      AF     Asia    Southern Asia
1   Åland Islands      AX   Europe  Northern Europe
2         Albania      AL   Europe  Southern Europe
3         Algeria      DZ   Africa  Northern Africa
4  American Samoa      AS  Oceania        Polynesia


In [4]:
# save the df
df_region.to_csv('../data/interim/region.csv', index=False)

In [5]:
# flag rows where any of the three columns is null
null_mask = df_region[['alpha-2','region','sub-region']].isnull().any(axis=1)

# slice out the bad (?) rows
bad_row = df_region.loc[null_mask].copy()
bad_row['null_columns'] = bad_row.apply(
    lambda row: [
        col for col in ['alpha-2','region','sub-region']
        if pd.isnull(row[col])
        ],
    axis=1
)

print(bad_row[['name','null_columns']])

                          name          null_columns
8                   Antarctica  [region, sub-region]
153                    Namibia             [alpha-2]
217  Taiwan, Province of China  [region, sub-region]


In [6]:
# ensure independence
df_region_corrected = df_region.copy()

# for Antarctica
df_region_corrected.loc[
    df_region_corrected['name'] == 'Antarctica',
    ['region', 'sub-region']
] = ['Antarctica', 'Antarctica']

# for Namibia
df_region_corrected.loc[
    df_region_corrected['name'] == 'Namibia',
    'alpha-2'
] = 'NA'

# for Taiwan
df_region_corrected.loc[
    df_region_corrected['name'] == 'Taiwan, Province of China',
    ['region', 'sub-region']
] = ['Asia', 'Eastern Asia']

print(df_region_corrected.loc[
    df_region_corrected['name'].isin([
        'Antarctica',
        'Namibia',
        'Taiwan, Province of China'
    ])
])

                          name alpha-2      region          sub-region
8                   Antarctica      AQ  Antarctica          Antarctica
153                    Namibia      NA      Africa  Sub-Saharan Africa
217  Taiwan, Province of China      TW        Asia        Eastern Asia


In [7]:
# save the corrected df
df_region_corrected.to_csv('../data/interim/region_corrected.csv', index=False)

In [8]:
# ensure independence
df_market_region = df_region_corrected.copy()

# group by region and collect unique sub-regions
subregions_by_region = (df_market_region.groupby('region')['sub-region'].unique().sort_index())

for region, subregions in subregions_by_region.items():
    print(region)
    for sub in subregions:
        print(f"    {sub}")
    print()

Africa
    Northern Africa
    Sub-Saharan Africa

Americas
    Latin America and the Caribbean
    Northern America

Antarctica
    Antarctica

Asia
    Southern Asia
    Western Asia
    South-eastern Asia
    Eastern Asia
    Central Asia

Europe
    Northern Europe
    Southern Europe
    Western Europe
    Eastern Europe

Oceania
    Polynesia
    Australia and New Zealand
    Melanesia
    Micronesia



In [9]:
import numpy as np

# map sub-regions per market
apac_subregions = [
    'Eastern Asia', 'South-eastern Asia', 'Southern Asia',
    'Central Asia', 'Western Asia',
    'Polynesia', 'Australia and New Zealand',
    'Melanesia', 'Micronesia'
]
latam_subregions = ['Latin America and the Caribbean']
na_subregions   = ['Northern America']
emea_subregions = [
    'Northern Europe', 'Southern Europe', 'Western Europe', 'Eastern Europe',
    'Northern Africa', 'Sub-Saharan Africa'
]

# conditions and corresponding market-region
conditions = [
    df_market_region['sub-region'].isin(apac_subregions),
    df_market_region['sub-region'].isin(latam_subregions),
    df_market_region['sub-region'].isin(na_subregions),
    df_market_region['sub-region'].isin(emea_subregions)
]
choices = [
    'APAC',
    'LATAM',
    'NA',
    'EMEA'
]

# create new column
df_market_region['market-region'] = np.select(conditions, choices, default='Other') # this one should be for Antarctica

print(df_market_region.head())

             name alpha-2   region       sub-region market-region
0     Afghanistan      AF     Asia    Southern Asia          APAC
1   Åland Islands      AX   Europe  Northern Europe          EMEA
2         Albania      AL   Europe  Southern Europe          EMEA
3         Algeria      DZ   Africa  Northern Africa          EMEA
4  American Samoa      AS  Oceania        Polynesia          APAC


In [10]:
# check the unique values to confirm
print(df_market_region['market-region'].unique())

['APAC' 'EMEA' 'LATAM' 'Other' 'NA']


In [11]:
# check which countries have the 'Other' market-region
other_countries = df_market_region[df_market_region['market-region'] == 'Other']

print(other_countries[['name', 'region', 'sub-region']])

         name      region  sub-region
8  Antarctica  Antarctica  Antarctica


In [12]:
# save the df
df_market_region.to_csv('../data/processed/market_region.csv', index=False)
df_market_region.to_excel('../data/processed/market_region.xlsx', index=False)